# Introduction to Antidromic Stimulation

## The Problem: Classifying Neurons by Their Projection Targets

Understanding brain function requires knowing not just *where* neurons are located, but *where they send their outputs*. In motor cortex, for example, some neurons project directly to the spinal cord to control movement (pyramidal tract neurons, PTNs), while others project to the basal ganglia to coordinate motor planning (corticostriatal neurons, CSNs). These cell types have distinct physiological properties and are differentially affected by neurological diseases like Parkinson's disease.

But here's the challenge: when you insert a recording electrode into motor cortex, you can only "see" the neuron's cell body and nearby processes. How do you determine where that neuron's axon goes - potentially tens of millimeters away?

Several methods exist for neuron classification:
- **Anatomical tracing**: Inject tracers at target sites; neurons that project there become labeled
- **Optogenetics**: Express light-sensitive proteins in specific projection classes
- **Antidromic stimulation**: Stimulate at potential target sites and look for characteristic responses

This tutorial focuses on **antidromic stimulation**, the classical electrophysiological approach used extensively in primate neurophysiology.

## Orthodromic vs Antidromic: The Direction of Action Potentials

Under normal physiological conditions, action potentials travel **orthodromically** - from the cell body (soma), down the axon, to the synaptic terminals at distant targets. This is the "forward" direction that carries information from the neuron to its downstream partners.

However, axons can conduct action potentials in either direction. If you artificially initiate an action potential at the axon terminal (by electrical stimulation), it will travel **antidromically** - backwards up the axon toward the cell body. When this antidromic spike reaches the soma, it can be detected by a recording electrode positioned there.

```
Orthodromic (normal physiology):     Cell Body ---> Axon ---> Target
Antidromic (experimental):           Cell Body <--- Axon <--- [STIM]
```

This bidirectionality is the key insight: if you stimulate at a particular brain region and a recorded neuron responds antidromically, you know that neuron's axon passes through (or terminates in) that region.

## A Specific Example: The Turner Lab Parkinson's Disease Study

To illustrate antidromic stimulation with real data, we will use recordings from a study by Pasquereau & Turner (2011, *Cerebral Cortex* 21:1362-1378). This research used antidromic identification to compare the activity of pyramidal tract neurons (PTNs) and corticostriatal neurons (CSNs) in monkeys before and after inducing parkinsonism with MPTP. By separating these cell types, the researchers discovered that Parkinson's disease pathology selectively affects PTNs while relatively sparing CSNs - a finding that would have been impossible without projection-based classification.

## Anatomical Context: M1, Cerebral Peduncle, and Striatum

In the Pasquereau & Turner study, recordings were made from **primary motor cortex (M1)** - the region of cortex most directly involved in generating voluntary movement commands. M1 contains multiple types of projection neurons, but two major classes were of interest:

### Pyramidal Tract Neurons (PTNs)
- **Location in cortex**: Layer 5b of M1 (deep layer, large pyramidal cells)
- **Projection target**: Spinal cord, via the corticospinal (pyramidal) tract
- **Axon pathway**: Axons descend through the corona radiata, enter the internal capsule, and converge in the **cerebral peduncle** - a massive white matter bundle at the base of the midbrain that carries all descending cortical motor commands
- **Function**: Direct control of spinal motor neurons and interneurons; execution of fine, skilled movements
- **Stimulation site for identification**: Cerebral peduncle (where PTN axons are densely bundled together)

### Corticostriatal Neurons (CSNs)
- **Location in cortex**: Layer 5 of M1 (intratelencephalic type)
- **Projection target**: Striatum (specifically the putamen, the motor division of the basal ganglia)
- **Axon pathway**: Axons project to the ipsilateral **posterolateral putamen**, which receives topographically organized inputs from motor cortices
- **Function**: Provide cortical information to basal ganglia circuits involved in action selection, motor learning, and movement vigor
- **Stimulation site for identification**: Posterolateral putamen (striatum)

By placing stimulating electrodes in both the cerebral peduncle and the putamen, researchers can determine which neurons are PTNs (respond to peduncle stimulation), which are CSNs (respond to putamen stimulation), and which are neither or both.

## Let's See an Antidromic Response

Before diving into technical details, let's look at what an antidromic response actually looks like. The data from the Pasquereau & Turner study has been publicly released on [DANDI Archive (Dandiset 001636)](https://dandiarchive.org/dandiset/001636/draft). We'll stream it directly and plot a single sweep from an antidromic stimulation experiment.

The session we'll use contains a corticostriatal neuron (CSN) that was identified by antidromic stimulation from the striatum.

In [ ]:
import h5py
import remfile
import numpy as np
import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient

from notebook_helpers import get_sweep_waveforms, measure_response_latency, measure_response_amplitude

# Connect to DANDI and get the dandiset
dandiset_id = "001636"
client = DandiAPIClient()
dandiset = client.get_dandiset(dandiset_id, "draft")

# Select a session with a corticostriatal neuron
asset_path = "sub-V/sub-V_ses-V++v4806++PostMPTP++Depth17700um++20000204_behavior+ecephys.nwb"
assets = list(dandiset.get_assets())
asset = next(a for a in assets if a.path == asset_path)
print(f"Streaming: {asset.path}")

# Stream the NWB file directly from DANDI (no download required)
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")

io = NWBHDF5IO(file=file)
nwbfile = io.read()
print(f"Session: {nwbfile.session_id}")

In [ ]:
# Get the antidromic sweeps and neuron data
units_df = nwbfile.units.to_dataframe()
sweeps_df = nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()
antidromic_latency = units_df['antidromic_latency_ms'].iloc[0]

# Plot a single sweep
sweep = sweeps_df.iloc[0]
time_ms, response_uv, stim_ua = get_sweep_waveforms(sweep)

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Top panel: Neural response
axes[0].plot(time_ms, response_uv, 'b-', linewidth=0.8)
axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
axes[0].axvline(antidromic_latency, color='green', linestyle=':', linewidth=2, label=f'Antidromic response ({antidromic_latency} ms)')
axes[0].set_ylabel('Neural Response (uV)', fontsize=12)
axes[0].set_title('One Antidromic Sweep', fontsize=14, fontweight='bold')
axes[0].legend(loc='upper right', fontsize=10)
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)
axes[0].text(-20, axes[0].get_ylim()[1] * 0.8, 'Recording in\nMotor Cortex', fontsize=14, fontweight='bold', va='top')

# Bottom panel: Stimulation current
axes[1].plot(time_ms, stim_ua, 'r-', linewidth=0.8)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Time relative to stimulation (ms)', fontsize=12)
axes[1].set_ylabel('Stimulation Current (uA)', fontsize=12)
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].text(-20, axes[1].get_ylim()[1] * 0.8, 'Stimulus in\nStriatum', fontsize=14, fontweight='bold', va='top')

plt.tight_layout()
plt.show()

### Understanding What We See

**That's it.** That's an antidromic response. The method is conceptually simple:
1. Stimulate at a potential axon target (here, the striatum)
2. Record at the cell body (here, motor cortex)
3. If the neuron's axon passes through the stimulation site, you see a response

**Top panel - Neural Response:**
- At **t = 0 ms** (red dashed line): A large, sharp deflection called the **stimulation artifact**. This is electrical contamination from the stimulation pulse - NOT neural activity.
- At **t = 6.4 ms** (green dotted line): A smaller but distinct waveform - the **antidromic spike**. This is the action potential that traveled backwards up the axon from the striatum to the motor cortex cell body.

**Bottom panel - Stimulation Current:**
- The **biphasic waveform** shows the current delivered: first positive, then negative (or vice versa)
- This charge-balanced pulse minimizes tissue damage during repeated stimulation

#### What is Latency?

The **latency** is the time between the stimulation and the arrival of the antidromic spike at the cell body. Here, it's 6.4 ms.

```
Latency = Distance / Conduction Velocity
```

This latency reflects:
- **Distance**: How far the action potential must travel (striatum to motor cortex)
- **Conduction velocity**: How fast action potentials propagate along this axon

#### Latency and Cell Type Classification

Different neuron types have different latencies because they have different axon properties:

| Cell Type | Stimulation Site | Typical Latency | Why |
|-----------|-----------------|-----------------|-----|
| **PTN** | Cerebral peduncle | 0.6-2.5 ms | Large, heavily myelinated axons = fast conduction |
| **CSN** | Striatum | 2-20 ms | Smaller axons = slower conduction |

The longer latency of this neuron (6.4 ms) is consistent with a corticostriatal neuron, not a pyramidal tract neuron.

#### Why Biphasic Stimulation?

The bottom panel shows a biphasic (two-phase) current pulse. This is standard for neural stimulation because:

| Issue | Monophasic (one phase) | Biphasic (two phases) |
|-------|----------------------|---------------------|
| **Charge accumulation** | Net charge builds up at electrode | Positive and negative cancel out |
| **Tissue damage** | pH changes, toxic byproducts | Minimal - charge balanced |
| **Electrode corrosion** | Electrochemical degradation | Reduced - no net current flow |

## Why Do We Need Validation?

The sweep above looks convincing - there's clearly a response at a consistent latency after stimulation. But how do we **know** this is an antidromic response?

The problem is that electrical stimulation can activate neurons through multiple pathways:

1. **Antidromic** (what we want): The stimulus directly activates the recorded neuron's axon, and the spike travels backward to the cell body
2. **Synaptic/Orthodromic** (confounding): The stimulus activates other neurons at the stimulation site, which then synaptically excite the recorded neuron through a forward pathway

Both would produce a spike after stimulation. We need rigorous criteria to distinguish them.

## The Three Criteria for Antidromic Activation

The electrophysiology literature has established three standard criteria for confirming antidromic activation (Fuller & Schlag, 1976, *Brain Research* 112:283-298):

| Criterion | Test | Why It Works |
|-----------|------|-------------|
| **Fixed latency** | Response occurs at constant delay across trials | Axon conduction time is invariant; synaptic delays fluctuate |
| **Frequency following** | Neuron follows stimulation at >200 Hz | Axons can fire rapidly; synapses fatigue at high frequencies |
| **Collision test** | Spontaneous spike blocks the antidromic response | Two action potentials annihilate when they meet on the same axon |

The **collision test** is the gold standard because it's the most definitive - it can only happen if the orthodromic and antidromic spikes are traveling on the same axon.

Let's illustrate each criterion with real data.

---

## Criterion 1: Fixed Latency

The first criterion tests whether the response latency is constant across trials. Axon conduction time is determined by physical properties (axon length and conduction velocity) that don't change from trial to trial. Synaptic transmission, in contrast, is probabilistic and introduces variable delays.

Let's overlay multiple sweeps to see if the latency is fixed:

In [ ]:
# Get all sweeps (both protocols demonstrate fixed latency)
all_sweeps = sweeps_df.copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: Overlay all sweeps
ax = axes[0]
for i in range(len(all_sweeps)):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.6, alpha=0.4)

ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2, 
           label=f'Expected latency ({antidromic_latency} ms)')
ax.set_xlim(-2, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=12)
ax.set_ylabel('Neural Response (uV)', fontsize=12)
ax.set_title(f'All Sweeps Overlaid (n={len(all_sweeps)})', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right panel: Waterfall view showing individual sweeps
ax = axes[1]
offset = 0
spacing = 60  # uV between traces

for i in range(min(15, len(all_sweeps))):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv + offset, 'b-', linewidth=0.8)
    ax.text(-3, offset, f'#{i+1}', fontsize=8, va='center')
    offset += spacing

ax.axvline(0, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=1.5, alpha=0.7)
ax.set_xlim(-5, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=12)
ax.set_title('Individual Sweeps (Waterfall View)', fontsize=13, fontweight='bold')
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

### What We See: Evidence for Fixed Latency

**Left panel (Overlay):**
- All sweeps are plotted on top of each other
- The responses form a **tight cluster** at exactly 6.4 ms after stimulation
- If latency were variable, we would see the responses spread out horizontally

**Right panel (Waterfall):**
- Each sweep is plotted separately, offset vertically
- You can trace a vertical line at 6.4 ms and see that **every sweep** has its response aligned
- This consistency is the hallmark of antidromic activation

Let's quantify this by measuring the actual latency in each sweep:

In [ ]:
# Measure latency from all sweeps
measured_latencies = []

for i in range(len(all_sweeps)):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    lat = measure_response_latency(time_ms, response_uv, antidromic_latency)
    if not np.isnan(lat):
        measured_latencies.append(lat)

# Plot latency histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Left: Zoomed view of response window
ax = axes[0]
for i in range(len(all_sweeps)):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.5, alpha=0.4)

ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2)
ax.axvspan(antidromic_latency - 0.5, antidromic_latency + 0.5, alpha=0.2, color='green',
           label='Expected +/- 0.5 ms')
ax.set_xlim(antidromic_latency - 3, antidromic_latency + 3)
ax.set_xlabel('Time (ms)', fontsize=11)
ax.set_ylabel('Response (uV)', fontsize=11)
ax.set_title('Zoomed View of Response Window', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right: Latency histogram
ax = axes[1]
ax.hist(measured_latencies, bins=15, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2,
           label=f'Expected: {antidromic_latency} ms')
ax.axvline(np.mean(measured_latencies), color='red', linestyle='--', linewidth=2,
           label=f'Measured mean: {np.mean(measured_latencies):.2f} ms')
ax.set_xlabel('Measured Latency (ms)', fontsize=11)
ax.set_ylabel('Count', fontsize=11)
latency_std = np.std(measured_latencies)
ax.set_title(f'Latency Distribution (SD = {latency_std:.3f} ms)', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

print(f"Latency statistics from {len(measured_latencies)} sweeps:")
print(f"  Mean:     {np.mean(measured_latencies):.3f} ms")
print(f"  SD:       {np.std(measured_latencies):.3f} ms")
print(f"  Range:    {np.min(measured_latencies):.3f} - {np.max(measured_latencies):.3f} ms")

### Interpreting the Fixed Latency Results

The histogram shows that measured latencies cluster tightly around the expected value:

- **SD < 1 ms**: This low variability is characteristic of antidromic activation
- **Tight, unimodal distribution**: All responses occur at essentially the same time

For comparison:
| Activation Type | Typical Latency SD |
|-----------------|-------------------|
| **Antidromic** | < 0.5 ms |
| **Monosynaptic** | 0.5 - 1.5 ms |
| **Polysynaptic** | 1.5 - 5+ ms |

**Conclusion**: The fixed latency criterion is satisfied.

---

## Criterion 2: The Collision Test (Gold Standard)

The collision test is the most definitive verification of antidromic activation. It exploits a fundamental property of axons: **two action potentials traveling in opposite directions on the same axon will collide and annihilate each other**.

### How Collision Works

```
Normal antidromic response (no collision):
    
    Cell Body <----[antidromic spike]---- [STIM at striatum]
         |
         V
    Recorded spike


Collision (spontaneous spike blocks response):

    Cell Body ---[spontaneous spike]---> 
                                    COLLISION!
              <---[antidromic spike]--- [STIM]
         |
         V
    NO recorded spike (blocked)
```

### The Critical Collision Window

For collision to occur, the spontaneous spike must be traveling down the axon when the antidromic spike is traveling up. This defines a **critical window** before stimulation:

```
Critical window = Antidromic latency + Axonal refractory period
```

If a spontaneous spike occurs within this window before stimulation, the antidromic response should be blocked.

### A Better Example for Collision

To clearly demonstrate collision, we'll load a different session that shows more dramatic amplitude variability between blocked and unblocked responses:

In [ ]:
# Load a different session with clearer collision effects
collision_asset_path = "sub-V/sub-V_ses-V++v4518++PostMPTP++Depth19900um++20000124_behavior+ecephys.nwb"
collision_asset = next(a for a in assets if a.path == collision_asset_path)
print(f"Loading collision example: {collision_asset.path.split('/')[-1]}")

collision_s3_url = collision_asset.get_content_url(follow_redirects=1, strip_query=False)
collision_file = h5py.File(remfile.File(collision_s3_url), mode="r")
collision_io = NWBHDF5IO(file=collision_file)
collision_nwbfile = collision_io.read()

# Get data from this session
collision_units_df = collision_nwbfile.units.to_dataframe()
collision_sweeps_df = collision_nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()

collision_latency = collision_units_df['antidromic_latency_ms'].iloc[0]
print(f"Neuron type: {collision_units_df['neuron_projection_type'].iloc[0]}")
print(f"Antidromic latency: {collision_latency} ms")

# Get collision sweeps
collision_sweeps = collision_sweeps_df[collision_sweeps_df['stimulation_protocol'] == 'Collision']
print(f"Collision sweeps: {len(collision_sweeps)}")

# Calculate critical window
refractory_period = 2.0  # ms
critical_window = collision_latency + refractory_period
print(f"Critical collision window: {critical_window:.1f} ms")

# Measure response amplitude for each sweep
amplitudes = [measure_response_amplitude(*get_sweep_waveforms(collision_sweeps.iloc[i])[:2], collision_latency) 
              for i in range(len(collision_sweeps))]
median_amp = np.median(amplitudes)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: All sweeps with critical window
ax = axes[0]
ax.axvspan(-critical_window, 0, alpha=0.2, color='orange', label=f'Critical window ({critical_window:.1f} ms)')

for i in range(len(collision_sweeps)):
    sweep = collision_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.5, alpha=0.3)

ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2, 
           label=f'Expected response ({collision_latency} ms)')
ax.set_xlim(-15, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=12)
ax.set_ylabel('Neural Response (uV)', fontsize=12)
ax.set_title(f'All Collision Sweeps (n={len(collision_sweeps)})', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right panel: Response amplitude distribution
ax = axes[1]
ax.hist(amplitudes, bins=12, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(median_amp, color='red', linestyle='--', linewidth=2, 
           label=f'Median: {median_amp:.1f} uV')
ax.set_xlabel('Response Amplitude (uV)', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Response Amplitude Distribution', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

amp_cv = np.std(amplitudes) / np.mean(amplitudes) * 100
print(f"\nResponse amplitude variability: CV = {amp_cv:.1f}%")
print(f"Range: {np.min(amplitudes):.1f} - {np.max(amplitudes):.1f} uV")

### Interpreting the Collision Test Results

**Left panel (All Sweeps):**
- The **orange shaded region** shows the critical collision window before stimulation
- Looking at the responses around the expected latency, some sweeps have **strong deflections** while others appear **weaker or absent**
- This variability is exactly what we expect from a collision test

**Right panel (Amplitude Histogram):**
- The histogram shows a **wide spread** of response amplitudes
- High amplitude = no collision (antidromic spike reached cell body)
- Low amplitude = likely collision (spontaneous spike blocked the response)

Let's separate the sweeps by amplitude to see this more clearly:

In [ ]:
# Separate sweeps by amplitude (above vs below median)
high_amp_indices = [i for i, amp in enumerate(amplitudes) if amp >= median_amp]
low_amp_indices = [i for i, amp in enumerate(amplitudes) if amp < median_amp]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: High amplitude sweeps (clear responses - no collision)
ax = axes[0]
ax.axvspan(-critical_window, 0, alpha=0.15, color='orange')

for idx in high_amp_indices:
    sweep = collision_sweeps.iloc[idx]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.6, alpha=0.5)

ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2)
ax.set_xlim(-15, 15)
ax.set_xlabel('Time (ms)', fontsize=11)
ax.set_ylabel('Response (uV)', fontsize=11)
ax.set_title(f'Clear Responses (n={len(high_amp_indices)}) - No Collision', fontsize=13, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.text(0.02, 0.98, 'No spontaneous spike\nin critical window\n-> Response present',
        transform=ax.transAxes, fontsize=9, va='top',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

# Right: Low amplitude sweeps (potential collisions)
ax = axes[1]
ax.axvspan(-critical_window, 0, alpha=0.15, color='orange')

for idx in low_amp_indices:
    sweep = collision_sweeps.iloc[idx]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.6, alpha=0.5)

ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2)
ax.set_xlim(-15, 15)
ax.set_xlabel('Time (ms)', fontsize=11)
ax.set_ylabel('Response (uV)', fontsize=11)
ax.set_title(f'Reduced Responses (n={len(low_amp_indices)}) - Potential Collisions', fontsize=13, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.text(0.02, 0.98, 'Spontaneous spike in\ncritical window?\n-> Response blocked',
        transform=ax.transAxes, fontsize=9, va='top',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

### The Collision Effect Visualized

**Left panel (Clear Responses):**
- These sweeps have **high amplitude** responses at the expected latency
- No spontaneous spike occurred in the critical window
- The antidromic spike traveled unimpeded to the cell body

**Right panel (Reduced Responses):**
- These sweeps have **low amplitude** or nearly absent responses
- A spontaneous spike likely occurred in the critical window and collided with the antidromic spike

**Why this is definitive proof:**

This can only happen if the orthodromic and antidromic spikes are on the **same axon**. If the neuron were being activated synaptically, spontaneous spikes would NOT block the response - they travel on different paths.

**Conclusion**: The collision test criterion is satisfied.

---

## Sessions Used in This Tutorial

We demonstrated the antidromic criteria using two sessions from the same dataset, each chosen for clarity:

| Criterion | Session | Why This Session |
|-----------|---------|------------------|
| **Fixed Latency** | v4806 | 31 sweeps with consistent responses |
| **Collision Test** | v4518 | 25 collision sweeps with high amplitude variability |

This illustrates a key advantage of standardized data formats (NWB) and public repositories (DANDI): you can easily access multiple datasets to find the clearest examples.

## PTN vs CSN: Two Projection Types

The neurons we examined were corticostriatal neurons (CSNs). For comparison, here's how PTNs and CSNs differ:

| Property | Pyramidal Tract Neurons (PTNs) | Corticostriatal Neurons (CSNs) |
|----------|-------------------------------|--------------------------------|
| **Projection target** | Spinal cord via cerebral peduncle | Striatum (putamen) |
| **Stimulation site** | Cerebral peduncle | Posterolateral putamen |
| **Antidromic latency** | 0.6-2.5 ms | 2-20 ms |
| **Axon diameter** | Large (fast conduction) | Smaller (slower conduction) |
| **Cortical layer** | Layer 5b (large pyramidal) | Layer 5 (intratelencephalic) |
| **Function** | Direct motor commands to spinal cord | Cortical input to basal ganglia |

In [ ]:
# Display classification for the first neuron we examined
unit = units_df.iloc[0]

print("Example neuron classification (from session v4806):")
print(f"  Type: {unit['neuron_projection_type']}")
print(f"  Stimulation site: {unit['antidromic_stimulation_sites']}")
print(f"  Latency: {unit['antidromic_latency_ms']} ms")

# Estimate conduction velocity (approximate distance M1 to striatum: ~30 mm)
distance_mm = 30
conduction_velocity = distance_mm / unit['antidromic_latency_ms']
print(f"  Estimated conduction velocity: ~{conduction_velocity:.1f} m/s")

## Summary

**What we learned:**

1. **Antidromic stimulation** classifies neurons by their axonal targets: stimulate at a potential target, record at the cell body, and look for backwards-traveling action potentials

2. **Two validation criteria** distinguish antidromic from synaptic activation:
   - **Fixed latency**: Axon conduction time is invariant; synaptic delays fluctuate
   - **Collision test**: Spontaneous and antidromic spikes annihilate on the same axon (gold standard)

3. **Cell type classification** follows from stimulation site:
   - Response to peduncle stimulation = Pyramidal Tract Neuron (PTN)
   - Response to striatum stimulation = Corticostriatal Neuron (CSN)

4. **In NWB format**, antidromic data is stored in:
   - `units` table: Classification results (type, latency, threshold)
   - `AntidromicSweepsIntervals`: Raw waveforms for each test sweep

**Further reading:**
- Pasquereau B, Turner RS (2011) Primary motor cortex of the parkinsonian monkey. *Cerebral Cortex* 21:1362-1378.
- Fuller JH, Schlag JD (1976) Determination of antidromic excitation by the collision test. *Brain Research* 112:283-298.